In [1]:
from dreye.api.optimize import lsq_linear as lsq
import numpy as np
import cvxpy as cp

In [2]:
n, m = 4, 6
A = np.random.random((n, m))
xtarget = np.ones(m)/2
#np.random.random(m)
qtarget = A @ xtarget
# qtarget = np.random.random((1, m))
etarget = qtarget / (1+qtarget)

In [3]:
x = cp.Variable((m), pos=True)

In [4]:
q = A @ x

In [5]:
e = q / (1+q)

In [6]:
e

Expression(QUASILINEAR, NONNEGATIVE, (4,))

In [7]:
cp.sum(q)/cp.sum(1+q)  # intensity

Expression(QUASILINEAR, NONNEGATIVE, ())

In [8]:
c = cp.Variable(1, pos=True)

In [9]:
q/cp.sum(q)

Expression(QUASILINEAR, NONNEGATIVE, (4,))

In [10]:
q == cp.sum(q) * qtarget/np.sum(qtarget)

Equality(Expression(AFFINE, NONNEGATIVE, (4,)), Expression(AFFINE, NONNEGATIVE, (4,)))

In [11]:
testc = cp.sum(q) == np.sum(qtarget)/np.sum(1+qtarget) * cp.sum(1+q)

In [25]:
testc.is_dqcp()

True

In [13]:
denom = cp.multiply((1+q), (1+qtarget))

In [14]:
num = q - qtarget

In [15]:
cp.sum(cp.abs(num) / denom)

Expression(UNKNOWN, NONNEGATIVE, ())

In [16]:
obj = cp.Minimize(cp.max(cp.abs(num)/denom))

In [17]:
constraints = [
    x <= 1
]

In [18]:
obj = cp.Minimize(cp.sum_squares(x - cp.sum(x)/x.size))
obj

Minimize(Expression(CONVEX, NONNEGATIVE, ()))

In [19]:
constraints = [
    x <= 1, 
    cp.max(cp.abs(num)/denom) <= 1e-4
]

In [18]:
prob = cp.Problem(obj, constraints)

In [19]:
prob

Problem(Minimize(Expression(QUASICONVEX, NONNEGATIVE, ())), [Inequality(Variable((6,), pos=True))])

In [20]:
prob.solve(qcp=True, verbose=True, solver=cp.SCS)

                                     CVXPY                                     
                                    v1.1.15                                    
(CVXPY) Dec 20 05:48:48 AM: Your problem has 6 variables, 1 constraints, and 0 parameters.
(CVXPY) Dec 20 05:48:48 AM: It is compliant with the following grammars: DQCP
(CVXPY) Dec 20 05:48:48 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Dec 20 05:48:48 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Dec 20 05:48:48 AM: Reducing DQCP problem to a one-parameter family of DCP problems, for bisection.

********************************************************************************
Preparing to bisect problem

minimize 0.0
subject to var0 <= 1.0
           var64 <= Promote(param52, (4,)) @ (Promote(1.0, (4,)) + [[0.44821144 0.60626252 0.41763696 0.03123655 0.0168976  0.62871017]
 [0.01335964 0.9948

1.2291978817513554e-07

In [21]:
np.round(x.value, 2)

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5])

In [22]:
xtarget

array([0.5, 0.5, 0.5, 0.5, 0.5, 0.5])